In [1]:
using LinearAlgebra
using Random

In [2]:
Random.seed!(0)
m,n = 30,40
A = randn(m,n)

30×40 Array{Float64,2}:
  0.679107   -0.762677   -0.0303032  …   0.511295    -1.34481    -0.458241
  0.828413    1.42305    -1.31924       -0.821429    -0.569668    2.19028
 -0.353007    0.408387   -0.482604      -0.246583    -2.70494    -1.94998
 -0.134854    0.588621   -1.15755       -0.326697     0.692166   -0.532182
  0.586617   -0.296278   -0.21415       -0.00636685  -1.27827     0.691888
  0.297336    0.691111   -1.37193    …  -0.971952    -0.116177   -0.58729
  0.0649475   0.506874   -1.53806       -0.131606    -0.121199    0.106751
 -0.109017   -0.0569299  -1.22742        1.1406       0.340587   -1.35188
 -0.51421    -1.77102    -1.95848        1.36668     -0.518573   -1.51645
  1.57433     1.59062    -0.473131      -1.11061     -0.353572    0.470316
 -0.688907    1.39706     1.33875    …   0.401544     0.557788    1.24658
 -0.762804    0.481556    0.807817       0.31931      1.40592    -0.7301
  0.397482   -0.321943   -1.05092        1.14304     -2.05401     0.0852577
  ⋮     

In [3]:
Random.seed!(0)

# target rank
r = 3
l = r+1 # sketch is just a tad larger than target rank
Ω = randn(n,l)

# randomized rangefinder
Y = A*Ω
factors = qr(Y)
Q = factors.Q[:,1:r]

30×3 Array{Float64,2}:
 -0.0884091  -0.170785     0.189007
  0.348881    0.00855675  -0.0316206
  0.057991    0.108026    -0.211148
  0.0524044   0.117656     0.129489
 -0.198116   -0.0756728    0.166696
 -0.278495   -0.0390785    0.14356
 -0.0929633   0.00245411  -0.170842
 -0.184657   -0.414333    -0.152659
  0.0653636   0.0854742    0.214072
  0.215567    0.375084     0.032067
  0.0124669   0.226661    -0.438028
 -0.111486   -0.0384466   -0.282908
  0.203644    0.0150304    0.286124
  ⋮                       
 -0.0936547   0.248655    -0.108251
 -0.260155    0.0765638    0.0493581
  0.106882   -0.221109    -0.29414
  0.201581    0.0111895    0.291133
 -0.115511    0.337995     0.0087112
 -0.193188   -0.120869     0.0933842
  0.0475955  -0.183511    -0.0500185
  0.277769   -0.11263      0.0571816
  0.133548   -0.0381261   -0.0883439
  0.226049   -0.226249     0.0383423
 -0.0892939  -0.148721     0.0934385
 -0.132403    0.310173     0.14478

In [4]:
# randomized SVD
C = Q'*A
factors = svd(C)
U,S,Vt = Q*factors.U, factors.S, factors.Vt
S[r+1:end] .= 0
print("Error in randomized rank r approximation is ", norm(U*diagm(S)*Vt - A))

Error in randomized rank r approximation is 32.77317174227981

In [5]:
# not much worse than the best possible 
print("Optimal error for any rank r factorization is ", norm(svd(A).S[r+1:end]))

Optimal error for any rank r factorization is 30.326373927281956

# single pass randomized SVD

In [6]:
# target rank
Random.seed!(0)
r = 3
l = 2r+1 # sketch is double target rank
k = 2l+1
Ω = randn(n,l)
Ψ = randn(k,m)

# form sketch
Y = A*Ω
W = Ψ*A

# recover
factors = qr(Y)
Q = factors.Q[:,1:l]
B = (Ψ*Q) \ W
factors = svd(B)
U,S,Vt = Q*factors.U, factors.S, factors.Vt
S[r+1:end] .= 0
println("Error in randomized rank r approximation is ", norm(U*diagm(S)*Vt - A))
println("Optimal error for any rank r factorization is ", norm(svd(A).S[r+1:end]))

Error in randomized rank r approximation is 40.49869287202217
Optimal error for any rank r factorization is 30.326373927281956


# Error as a function of oversampling

In [7]:
Random.seed!(0)

# target rank
r = 1
function rsvd_error(oversampling)
    l = r+oversampling # sketch is just a tad larger than target rank
    Ω = randn(n,l)

    # randomized rangefinder
    Y = A*Ω
    factors = qr(Y)
    Q = factors.Q[:,1:l]
    C = Q'*A
    factors = svd(C)
    U,S,Vt = Q*factors.U, factors.S, factors.Vt
    S[r+1:end] .= 0
    return norm(U*diagm(S)*Vt - A)
end

opt_error = norm(svd(A).S[r+1:end])
errors = [rsvd_error(extra) for extra=0:10] ./ opt_error

11-element Array{Float64,1}:
 1.0276610464624685
 1.017244067043012
 1.0199929413466724
 1.0208700283546948
 1.016168710454389
 1.0070504806973306
 1.0120167374067968
 1.0120131037388538
 1.0098980677111464
 1.011316346696152
 1.005723746059919

In [ ]:
using Plots
plot(errors)